# DeepLOB: Deep Convolutional Neural Networks for Limit Order Books

### Authors: Zihao Zhang, Stefan Zohren and Stephen Roberts
Oxford-Man Institute of Quantitative Finance, Department of Engineering Science, University of Oxford

This jupyter notebook is used to demonstrate our recent paper [2] published in IEEE Transactions on Singal Processing. We use FI-2010 [1] dataset and present how model architecture is constructed here. 

### Data:
The FI-2010 is publicly avilable and interested readers can check out their paper [1]. The dataset can be downloaded from: https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649 

Otherwise, the notebook will download the data automatically or it can be obtained from: 

https://drive.google.com/drive/folders/1Xen3aRid9ZZhFqJRgEMyETNazk02cNmv?usp=sharing.

### References:
[1] Ntakaris A, Magris M, Kanniainen J, Gabbouj M, Iosifidis A. Benchmark dataset for mid‐price forecasting of limit order book data with machine learning methods. Journal of Forecasting. 2018 Dec;37(8):852-66. https://arxiv.org/abs/1705.03233

[2] Zhang Z, Zohren S, Roberts S. DeepLOB: Deep convolutional neural networks for limit order books. IEEE Transactions on Signal Processing. 2019 Mar 25;67(11):3001-12. https://arxiv.org/abs/1808.03668

### This notebook runs on tensorflow 2.

In [1]:
# obtain data
import os 
if not os.path.isfile('data.zip'):
    !wget https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
    !unzip -n data.zip
    print('data downloaded.')
else:
    print('data already existed.')

data already existed.


In [2]:
# limit gpu memory
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# load packages
import pandas as pd
import pickle
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# set random seeds
np.random.seed(1)
tf.random.set_seed(2)


# Data preparation

We used no auction dataset that is normalised by decimal precision approach in their work. The first 40 columns of the FI-2010 dataset are 10 levels ask and bid information for a limit order book and we only use these 40 features in our network. The last 5 columns of the FI-2010 dataset are the labels with different prediction horizons. 

In [4]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)
    dY = np.array(Y)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]
    return dataX.reshape(dataX.shape + (1,)), dataY

def prepare_x_y(data, k, T):
    x = prepare_x(data)
    y = get_label(data)
    x, y = data_classification(x, y, T=T)
    y = y[:,k] - 1
    y = to_categorical(y, 3)
    return x, y

In [5]:
# please change the data_path to your local path
# data_path = '/nfs/home/zihaoz/limit_order_book/data'

dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

k = 4 # which prediction horizon
T = 100 # the length of a single input
n_hiddens = 64
checkpoint_filepath = './model_tensorflow2/weights'

trainX_CNN, trainY_CNN = prepare_x_y(dec_train, k, T)
valX_CNN, valY_CNN = prepare_x_y(dec_val, k, T)
testX_CNN, testY_CNN = prepare_x_y(dec_test, k, T)

print(trainX_CNN.shape, trainY_CNN.shape)
print(valX_CNN.shape, valY_CNN.shape)
print(testX_CNN.shape, testY_CNN.shape)

(203701, 100, 40, 1) (203701, 3)
(50851, 100, 40, 1) (50851, 3)
(139488, 100, 40, 1) (139488, 3)


In [6]:
def get_image_data(cnn_data):
    new_x = cnn_data.copy()
    for j in range(new_x.shape[0]):
        x_coords = np.arange(
            np.min(new_x[j,:,::2,0]), 
            np.max(new_x[j,:,::2,0]), 
            np.min( np.abs( np.diff( new_x[j,0,::4,0] ) ) ) #ticksize
        )
        new_image = np.zeros((100,len(x_coords)+1))
        
        for z in range(100):
            new_image[z, np.digitize( new_x[j,z,2::4,0], x_coords )] = new_x[j,z,3::4,0] #bid prices
            new_image[z, np.digitize( new_x[j,z,::4,0], x_coords )] = -new_x[j,z,1::4,0] #ask prices

        anchor_position = np.digitize( new_x[j,-1,2,0], x_coords )
        while anchor_position < 20:
            new_image = np.hstack([np.zeros((100,1)),new_image])
            anchor_position += 1

        while new_image.shape[1] < anchor_position+20:
            new_image = np.hstack([new_image,np.zeros((100,1))])
            
        new_x[j,:,:,0] = new_image[:,range(anchor_position-20,anchor_position+20)]
        if j % 100 == 0:
            print( j, end="\r")
    return new_x
    
new_x_train = get_image_data(trainX_CNN)
new_x_val = get_image_data(valX_CNN)
new_x_test = get_image_data(testX_CNN)

# Model Architecture

Please find the detailed discussion of our model architecture in our paper.

In [7]:
def create_deeplob(T, NF, number_of_lstm):
    input_lmd = Input(shape=(T, NF, 1))
    
    # build the convolutional block
    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 10))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    
    # build the inception module
    convsecond_1 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = Conv2D(64, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = Conv2D(64, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = Conv2D(64, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)
    
    convsecond_output = keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)
    conv_reshape = Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)
    conv_reshape = keras.layers.Dropout(0.2, noise_shape=(None, 1, int(conv_reshape.shape[2])))(conv_reshape, training=True)

    # build the last LSTM layer
    conv_lstm = LSTM(number_of_lstm)(conv_reshape)

    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model = Model(inputs=input_lmd, outputs=out)
    adam = keras.optimizers.Adam(lr=0.0001)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

deeplob = create_deeplob(trainX_CNN.shape[1], trainX_CNN.shape[2], n_hiddens)
deeplob.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 40, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 100, 20, 32)  96          ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 100, 20, 32)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 100, 20, 32)  4128        ['leaky_re_lu[0][0]']        

/home/jefferythewind/anaconda3/envs/ml/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Model Training

In [8]:
%%time

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

deeplob.fit(new_x_train, trainY_CNN, validation_data=(new_x_val, valY_CNN), 
            epochs=400, batch_size=128, verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/400
1592/1592 - 29s - loss: 0.8065 - accuracy: 0.6246 - val_loss: 0.6922 - val_accuracy: 0.7190 - 29s/epoch - 18ms/step
Epoch 2/400
1592/1592 - 24s - loss: 0.5709 - accuracy: 0.7726 - val_loss: 0.6253 - val_accuracy: 0.7500 - 24s/epoch - 15ms/step
Epoch 3/400
1592/1592 - 25s - loss: 0.5235 - accuracy: 0.7944 - val_loss: 0.5779 - val_accuracy: 0.7753 - 25s/epoch - 15ms/step
Epoch 4/400
1592/1592 - 25s - loss: 0.4965 - accuracy: 0.8077 - val_loss: 0.5471 - val_accuracy: 0.7867 - 25s/epoch - 15ms/step
Epoch 5/400
1592/1592 - 25s - loss: 0.4738 - accuracy: 0.8196 - val_loss: 0.5510 - val_accuracy: 0.7842 - 25s/epoch - 16ms/step
Epoch 6/400
1592/1592 - 25s - loss: 0.4582 - accuracy: 0.8272 - val_loss: 0.5096 - val_accuracy: 0.8085 - 25s/epoch - 16ms/step
Epoch 7/400
1592/1592 - 25s - loss: 0.4420 - accuracy: 0.8339 - val_loss: 0.4923 - val_accuracy: 0.8166 - 25s/epoch - 16ms/step
Epoch 8/400
1592/1592 - 25s - loss: 0.4292 - accuracy: 0.8400 - val_loss: 0.4848 - val_accuracy: 0.8211 

Epoch 70/400
1592/1592 - 25s - loss: 0.2356 - accuracy: 0.9167 - val_loss: 0.3126 - val_accuracy: 0.8886 - 25s/epoch - 16ms/step
Epoch 71/400
1592/1592 - 25s - loss: 0.2339 - accuracy: 0.9174 - val_loss: 0.3125 - val_accuracy: 0.8907 - 25s/epoch - 16ms/step
Epoch 72/400
1592/1592 - 25s - loss: 0.2330 - accuracy: 0.9174 - val_loss: 0.3155 - val_accuracy: 0.8870 - 25s/epoch - 16ms/step
Epoch 73/400
1592/1592 - 25s - loss: 0.2340 - accuracy: 0.9174 - val_loss: 0.3209 - val_accuracy: 0.8860 - 25s/epoch - 16ms/step
Epoch 74/400
1592/1592 - 25s - loss: 0.2311 - accuracy: 0.9184 - val_loss: 0.3039 - val_accuracy: 0.8910 - 25s/epoch - 16ms/step
Epoch 75/400
1592/1592 - 25s - loss: 0.2303 - accuracy: 0.9190 - val_loss: 0.3157 - val_accuracy: 0.8860 - 25s/epoch - 16ms/step
Epoch 76/400
1592/1592 - 25s - loss: 0.2313 - accuracy: 0.9184 - val_loss: 0.3110 - val_accuracy: 0.8889 - 25s/epoch - 16ms/step
Epoch 77/400
1592/1592 - 25s - loss: 0.2860 - accuracy: 0.8945 - val_loss: 0.3053 - val_accuracy:

Epoch 134/400
1592/1592 - 25s - loss: 0.1945 - accuracy: 0.9307 - val_loss: 0.3229 - val_accuracy: 0.8875 - 25s/epoch - 16ms/step
Epoch 135/400
1592/1592 - 25s - loss: 0.1950 - accuracy: 0.9302 - val_loss: 0.3271 - val_accuracy: 0.8852 - 25s/epoch - 16ms/step
Epoch 136/400
1592/1592 - 25s - loss: 0.1927 - accuracy: 0.9308 - val_loss: 0.3291 - val_accuracy: 0.8872 - 25s/epoch - 16ms/step
Epoch 137/400
1592/1592 - 25s - loss: 0.1931 - accuracy: 0.9310 - val_loss: 0.3274 - val_accuracy: 0.8874 - 25s/epoch - 16ms/step
Epoch 138/400
1592/1592 - 25s - loss: 0.1920 - accuracy: 0.9315 - val_loss: 0.3338 - val_accuracy: 0.8854 - 25s/epoch - 16ms/step
Epoch 139/400
1592/1592 - 25s - loss: 0.1908 - accuracy: 0.9316 - val_loss: 0.3340 - val_accuracy: 0.8870 - 25s/epoch - 16ms/step
Epoch 140/400
1592/1592 - 25s - loss: 0.1914 - accuracy: 0.9308 - val_loss: 0.3265 - val_accuracy: 0.8873 - 25s/epoch - 16ms/step
Epoch 141/400
1592/1592 - 25s - loss: 0.1909 - accuracy: 0.9321 - val_loss: 0.3313 - val_a

1592/1592 - 25s - loss: 0.1596 - accuracy: 0.9424 - val_loss: 0.3732 - val_accuracy: 0.8781 - 25s/epoch - 16ms/step
Epoch 203/400
1592/1592 - 25s - loss: 0.1593 - accuracy: 0.9423 - val_loss: 0.3860 - val_accuracy: 0.8743 - 25s/epoch - 16ms/step
Epoch 204/400
1592/1592 - 25s - loss: 0.1584 - accuracy: 0.9422 - val_loss: 0.3827 - val_accuracy: 0.8788 - 25s/epoch - 16ms/step
Epoch 205/400
1592/1592 - 25s - loss: 0.1583 - accuracy: 0.9431 - val_loss: 0.3909 - val_accuracy: 0.8766 - 25s/epoch - 16ms/step
Epoch 206/400
1592/1592 - 25s - loss: 0.1579 - accuracy: 0.9430 - val_loss: 0.3815 - val_accuracy: 0.8772 - 25s/epoch - 16ms/step
Epoch 207/400
1592/1592 - 25s - loss: 0.1565 - accuracy: 0.9433 - val_loss: 0.3881 - val_accuracy: 0.8765 - 25s/epoch - 16ms/step
Epoch 208/400
1592/1592 - 25s - loss: 0.1566 - accuracy: 0.9437 - val_loss: 0.3794 - val_accuracy: 0.8797 - 25s/epoch - 16ms/step
Epoch 209/400
1592/1592 - 25s - loss: 0.1569 - accuracy: 0.9434 - val_loss: 0.3917 - val_accuracy: 0.874

Epoch 266/400
1592/1592 - 25s - loss: 0.1326 - accuracy: 0.9515 - val_loss: 0.4504 - val_accuracy: 0.8705 - 25s/epoch - 16ms/step
Epoch 267/400
1592/1592 - 25s - loss: 0.1318 - accuracy: 0.9523 - val_loss: 0.4403 - val_accuracy: 0.8692 - 25s/epoch - 16ms/step
Epoch 268/400
1592/1592 - 25s - loss: 0.1318 - accuracy: 0.9521 - val_loss: 0.4416 - val_accuracy: 0.8711 - 25s/epoch - 16ms/step
Epoch 269/400
1592/1592 - 25s - loss: 0.1322 - accuracy: 0.9519 - val_loss: 0.4408 - val_accuracy: 0.8693 - 25s/epoch - 16ms/step
Epoch 270/400
1592/1592 - 25s - loss: 0.1311 - accuracy: 0.9525 - val_loss: 0.4435 - val_accuracy: 0.8687 - 25s/epoch - 16ms/step
Epoch 271/400
1592/1592 - 25s - loss: 0.1299 - accuracy: 0.9526 - val_loss: 0.4422 - val_accuracy: 0.8710 - 25s/epoch - 16ms/step
Epoch 272/400
1592/1592 - 25s - loss: 0.1291 - accuracy: 0.9529 - val_loss: 0.4488 - val_accuracy: 0.8705 - 25s/epoch - 16ms/step
Epoch 273/400
1592/1592 - 25s - loss: 0.1298 - accuracy: 0.9524 - val_loss: 0.4483 - val_a

1592/1592 - 25s - loss: 0.1107 - accuracy: 0.9596 - val_loss: 0.4933 - val_accuracy: 0.8693 - 25s/epoch - 16ms/step
Epoch 330/400
1592/1592 - 25s - loss: 0.1109 - accuracy: 0.9594 - val_loss: 0.5059 - val_accuracy: 0.8633 - 25s/epoch - 16ms/step
Epoch 331/400
1592/1592 - 25s - loss: 0.1108 - accuracy: 0.9598 - val_loss: 0.5021 - val_accuracy: 0.8661 - 25s/epoch - 16ms/step
Epoch 332/400
1592/1592 - 25s - loss: 0.1108 - accuracy: 0.9597 - val_loss: 0.4995 - val_accuracy: 0.8670 - 25s/epoch - 16ms/step
Epoch 333/400
1592/1592 - 25s - loss: 0.1101 - accuracy: 0.9603 - val_loss: 0.5013 - val_accuracy: 0.8678 - 25s/epoch - 16ms/step
Epoch 334/400
1592/1592 - 25s - loss: 0.1104 - accuracy: 0.9597 - val_loss: 0.4990 - val_accuracy: 0.8631 - 25s/epoch - 16ms/step
Epoch 335/400
1592/1592 - 25s - loss: 0.1087 - accuracy: 0.9603 - val_loss: 0.5022 - val_accuracy: 0.8664 - 25s/epoch - 16ms/step
Epoch 336/400
1592/1592 - 25s - loss: 0.1088 - accuracy: 0.9604 - val_loss: 0.5502 - val_accuracy: 0.849

Epoch 393/400
1592/1592 - 25s - loss: 0.0961 - accuracy: 0.9649 - val_loss: 0.5565 - val_accuracy: 0.8657 - 25s/epoch - 16ms/step
Epoch 394/400
1592/1592 - 25s - loss: 0.0936 - accuracy: 0.9662 - val_loss: 0.5510 - val_accuracy: 0.8655 - 25s/epoch - 16ms/step
Epoch 395/400
1592/1592 - 25s - loss: 0.0938 - accuracy: 0.9661 - val_loss: 0.5532 - val_accuracy: 0.8668 - 25s/epoch - 16ms/step
Epoch 396/400
1592/1592 - 25s - loss: 0.0942 - accuracy: 0.9656 - val_loss: 0.5692 - val_accuracy: 0.8633 - 25s/epoch - 16ms/step
Epoch 397/400
1592/1592 - 25s - loss: 0.0932 - accuracy: 0.9663 - val_loss: 0.5542 - val_accuracy: 0.8682 - 25s/epoch - 16ms/step
Epoch 398/400
1592/1592 - 25s - loss: 0.0914 - accuracy: 0.9666 - val_loss: 0.5660 - val_accuracy: 0.8615 - 25s/epoch - 16ms/step
Epoch 399/400
1592/1592 - 25s - loss: 0.0927 - accuracy: 0.9665 - val_loss: 0.5738 - val_accuracy: 0.8643 - 25s/epoch - 16ms/step
Epoch 400/400
1592/1592 - 25s - loss: 0.0942 - accuracy: 0.9660 - val_loss: 0.5569 - val_a

# Model Testing

In [11]:
deeplob.load_weights(checkpoint_filepath)
pred = deeplob.predict(new_x_test)

4359/4359 [==============================] - 16s 4ms/step


In [12]:
print('accuracy_score:', accuracy_score(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1)))
print(classification_report(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1), digits=4))

accuracy_score: 0.899260151410874
              precision    recall  f1-score   support

           0     0.8934    0.9162    0.9046     47915
           1     0.9028    0.8781    0.8903     48050
           2     0.9021    0.9040    0.9030     43523

    accuracy                         0.8993    139488
   macro avg     0.8994    0.8994    0.8993    139488
weighted avg     0.8993    0.8993    0.8992    139488

